In [1]:
import pandas as pd
import requests
from sqlalchemy import create_engine

## Connecting to different url to obtain card name and price

In [2]:
Throne_of_Eldraine_url = "https://www.mtggoldfish.com/index/ELD#online"
Theros_Beyond_Death_url = "https://www.mtggoldfish.com/index/THB#online"
IKO_url = "https://www.mtggoldfish.com/index/IKO#online"
M20_url = "https://www.mtggoldfish.com/index/M20#online"
WAR_url = "https://www.mtggoldfish.com/index/WAR#online"
RNA_url = "https://www.mtggoldfish.com/index/RNA#online"
GRN_url = "https://www.mtggoldfish.com/index/GRN#online"

In [3]:
ELD_cards = pd.read_html(Throne_of_Eldraine_url)
TBD_cards = pd.read_html(Theros_Beyond_Death_url)
IKD_cards = pd.read_html(IKO_url)
M20_cards = pd.read_html(M20_url)
WAR_cards = pd.read_html(WAR_url)
RNA_cards = pd.read_html(RNA_url)
GRN_cards = pd.read_html(GRN_url)

In [4]:
ELD_cards = ELD_cards[1]
TBD_cards = TBD_cards[1]
IKD_cards = IKD_cards[1]
M20_cards = M20_cards[1]
WAR_cards = WAR_cards[1]
RNA_cards = RNA_cards[1]
GRN_cards = GRN_cards[1]

## Joining all of the Data Tables into One Complete Table

In [5]:
standard_set = [ELD_cards,TBD_cards, IKD_cards, M20_cards, WAR_cards, RNA_cards, GRN_cards]

In [6]:
standard_set = pd.concat(standard_set)

In [7]:
standard_set = standard_set.reset_index().drop(columns = ['index','Daily','Daily(%)','Weekly'])

In [8]:
standard_set.columns

Index(['Card', 'Set', 'Rarity', 'Price', 'Weekly (%)'], dtype='object')

In [9]:
standard_set.columns = map(str.lower, standard_set.columns)

In [10]:
connection_string = "postgres:postgres@localhost:5432/javascript_db"
engine = create_engine(f'postgresql://{connection_string}')

In [18]:
standard_set = standard_set.drop_duplicates(subset = 'card', keep = 'first')

In [19]:
card_list = standard_set.loc[:,['card', 'set', 'rarity', 'price']]

In [20]:
card_list = card_list.rename(columns = {"card":"card_name", "set":"card_set", "rarity":"card_rarity", 'price':'card_price'})

In [21]:
standard_set_transformed = card_list.copy()

In [22]:
standard_set_transformed.to_sql(name = 'card_list', con = engine, if_exists = 'append', index = False)